# CLEAR: Clustering and Stock Impact-based News Recommendation System

## 시스템 데모 노트북 (개선된 버전)

이 노트북은 CLEAR(Clustering and Stock Impact-based News Recommendation System) 시스템의 각 모듈을 단계별로 시연합니다. 네이버의 AiRs 아키텍처를 기반으로 하되, 개인화 대신 주가 영향에 중점을 둔 뉴스 추천 시스템입니다.

### 목차
1. 환경 설정 및 데이터 로드
2. 텍스트 전처리
3. 뉴스 벡터화
4. 뉴스 클러스터링
5. 주가 영향 분석 (개선된 버전)
6. 고급 주가 영향 분석 (수정 및 개선)
7. 뉴스 추천
8. 결과 시각화 및 평가

## 1. 환경 설정 및 데이터 로드

먼저 필요한 라이브러리를 가져오고 시스템 구성 요소를 초기화합니다.

In [1]:
# 필요한 라이브러리 가져오기
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import logging
warnings.filterwarnings('ignore')

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 한글 폰트 설정
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 경로 설정
current_dir = os.getcwd()
if current_dir.endswith('notebooks/demo'):
    sys.path.append('../../')
else:
    sys.path.append('./')

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8-whitegrid')

### CLEAR 모듈 가져오기

이제 CLEAR 시스템의 각 모듈을 가져옵니다. 개선된 고급 주가 영향 분석기를 포함합니다.

In [2]:
# CLEAR 모듈 가져오기
from src.data.text_preprocessor import TextPreprocessor
from src.models.news_vectorizer import NewsVectorizer
from src.models.news_clustering import NewsClustering
from src.models.stock_impact_analyzer import StockImpactAnalyzer
from src.models.advanced_stock_impact_analyzer import AdvancedStockImpactAnalyzer
from src.models.news_recommender import NewsRecommender
from src.evaluation import CLEAREvaluator

### 데이터 로드

제공된 뉴스 및 주가 데이터를 로드합니다.

### 주가 데이터 구조 설명

주가 데이터는 다음과 같은 구조를 가집니다:
- Date: 날짜 (YYYYMMDD 형식)
- Time: 시간 (이 열은 모두 0이므로 무시합니다)
- Start: 시가 (09:00 시장 개장 가격)
- High: 고가
- Low: 저가
- End: 종가 (15:00 시장 마감 가격)
- Volume: 거래량

주의: 주가 데이터는 일별 데이터이며, Time 열은 무시해야 합니다.

In [3]:
# 주가 데이터 로드
stock_data_path = os.path.join('data', 'stock', 'stockprice_005930.csv')
stock_df = pd.read_csv(stock_data_path)

# 날짜를 datetime으로 변환
stock_df['Date'] = pd.to_datetime(stock_df['Date'], format='%Y%m%d')

# Time 열 제거 (모두 0이므로 무시)
stock_df = stock_df.drop('Time', axis=1)

# 주가 데이터 확인
print(f"주가 데이터 기간: {stock_df['Date'].min()} ~ {stock_df['Date'].max()}")
print(f"총 거래일 수: {len(stock_df)}")
stock_df.head()

주가 데이터 기간: 2013-07-16 00:00:00 ~ 2025-03-05 00:00:00
총 거래일 수: 2856


,Date,Start,High,Low,End,Volume
0,2025-03-05,55100,55400,54200,54300,6367338
1,2025-03-04,53900,55000,53800,54500,18553933
2,2025-02-28,55400,55700,54500,54500,28036698
3,2025-02-27,56500,57100,56200,56300,14975356
4,2025-02-26,57000,57100,56100,56600,18117091


### 뉴스 데이터 구조 설명

뉴스 데이터는 다음과 같은 구조를 가집니다:
- Title: 뉴스 제목
- Date: 날짜 및 시간 (YYYYMMDD HH:MM 형식)
- Press: 언론사
- Link: 뉴스 URL
- Body: 뉴스 본문
- Emotion: 감정 정보
- Num_comment: 댓글 수
- AI Summary: AI 요약 (일부 뉴스에만 존재)

In [4]:
# 뉴스 데이터 로드
news_data_path = os.path.join('data', 'news', 'yna_005930_all.csv')
news_df = pd.read_csv(news_data_path)

# 날짜 형식 확인
print(f"날짜 형식 예시: {news_df['Date'].iloc[0]}")

# 뉴스 데이터 확인
print(f"뉴스 데이터 수: {len(news_df)}")
news_df[['Title', 'Date', 'Press']].head()

날짜 형식 예시: 20190101 15:36
뉴스 데이터 수: 32208


,Title,Date,Press
0,"세계 2위 휴대전화 시장 인도, 이제 중·고가 제품에 눈 돌린다",20190101 15:36,yna
1,[부고] 홍현칠(삼성전자 서남아총괄)씨 부친상,20190101 14:53,yna
2,"SK그룹, CES서 신사업 기회 모색…경영진 총출동",20190101 09:00,yna
3,'알박기 노조'에 미행까지…삼성에버랜드 노조방해 13명 기소,20190101 09:00,yna
4,CES 달굴 모바일은…LG·소니 중저가폰에 로욜 폴더블폰도,20190101 08:10,yna


## 2. 텍스트 전처리

뉴스 텍스트를 전처리하여 분석에 적합한 형태로 변환합니다. 한국어 텍스트 처리를 위한 특별한 설정을 포함합니다.

In [5]:
# 텍스트 전처리기 초기화
text_preprocessor = TextPreprocessor(
    language='ko',
    use_mecab=True,  # 한국어 형태소 분석기 사용
    remove_stopwords=True,
    custom_stopwords=['기자', '연합뉴스', '뉴스', '기사', '서울', '이미지', '확대', '제공']
)

# 뉴스 제목과 본문 전처리
news_df['processed_title'] = news_df['Title'].apply(text_preprocessor.preprocess_text)
news_df['processed_body'] = news_df['Body'].apply(text_preprocessor.preprocess_text)

# 전처리 결과 확인
for i in range(3):
    print(f"원본 제목: {news_df['Title'].iloc[i]}")
    print(f"전처리 제목: {news_df['processed_title'].iloc[i]}")
    print("-" * 80)

2025-03-31 01:35:23,911 - src.data.text_preprocessor - INFO - Initialized Mecab tokenizer
2025-03-31 01:35:23,986 - src.data.text_preprocessor - INFO - Initialized 405 stopwords
2025-03-31 01:35:23,987 - src.data.text_preprocessor - INFO - Initialized TextPreprocessor with language=ko, use_mecab=True


원본 제목: 세계 2위 휴대전화 시장 인도, 이제 중·고가 제품에 눈 돌린다
전처리 제목: 세계 휴대 전화 인도 이제 제품 돌린다
--------------------------------------------------------------------------------
원본 제목: [부고] 홍현칠(삼성전자 서남아총괄)씨 부친상
전처리 제목: 부고 홍현칠 삼성전자 총괄 부친상
--------------------------------------------------------------------------------
원본 제목: SK그룹, CES서 신사업 기회 모색…경영진 총출동
전처리 제목: 신사업 기회 모색 경영진 총출동
--------------------------------------------------------------------------------


### 텍스트 전처리 과정 설명

텍스트 전처리는 다음 단계로 이루어집니다:

1. **텍스트 정규화**: HTML 태그 제거, 특수 문자 제거, 소문자 변환 등
2. **토큰화**: 텍스트를 개별 토큰(단어)으로 분리
3. **불용어 제거**: 분석에 불필요한 일반적인 단어 제거
4. **형태소 분석**: 한국어 텍스트의 경우 Mecab을 사용하여 형태소 분석 수행

한국어 텍스트 처리를 위해 Mecab 형태소 분석기를 사용하여 더 정확한 단어 분리를 수행합니다.

## 3. 뉴스 벡터화

전처리된 뉴스 텍스트를 벡터로 변환합니다. 제목과 본문을 다르게 처리하는 옵션을 포함합니다.

In [6]:
# 뉴스 벡터화기 초기화
news_vectorizer = NewsVectorizer(
    method='tfidf',
    max_features=10000,
    embedding_dim=300,
    use_gpu=True,
    title_weight=2.0
)

# 뉴스 벡터화
combined_texts = [
    f"{title} {body}"
    for title, body in zip(news_df['processed_title'], news_df['processed_body'])
]

# Fit and transform using the combined texts
news_vectors = news_vectorizer.fit_transform(combined_texts)

# 벡터화 결과 확인
print(f"뉴스 벡터 형태: {news_vectors.shape}")
print(f"벡터 차원: {news_vectors.shape[1]}")

# 벡터를 DataFrame에 추가
news_df['vector'] = list(news_vectors)

2025-03-31 01:35:55,240 - src.models.news_vectorizer - WARNING - GPU requested but not available, falling back to CPU
2025-03-31 01:35:55,240 - src.models.news_vectorizer - INFO - Initialized TF-IDF vectorizer with max_features=10000
2025-03-31 01:35:55,241 - src.models.news_vectorizer - INFO - Initialized NewsVectorizer with method=tfidf, max_features=10000
2025-03-31 01:35:55,256 - src.models.news_vectorizer - INFO - Fitting tfidf vectorizer on 32208 documents
2025-03-31 01:36:05,852 - src.models.news_vectorizer - INFO - TF-IDF vectorizer fitted with vocabulary size: 10000
2025-03-31 01:36:09,433 - src.models.news_vectorizer - INFO - Saved TF-IDF vectorizer to /Users/hwangchiung/CLEAR/models/vectorizers/tfidf_vectorizer.joblib
2025-03-31 01:36:09,434 - src.models.news_vectorizer - INFO - Transforming 32208 documents using tfidf
2025-03-31 01:36:14,487 - src.models.news_vectorizer - INFO - Transformed 32208 documents to shape (32208, 10000)


뉴스 벡터 형태: (32208, 10000)
벡터 차원: 10000


### 뉴스 벡터화 과정 설명

뉴스 벡터화는 텍스트를 수치 벡터로 변환하는 과정입니다. 이 시스템에서는 다음 방법을 지원합니다:

1. **TF-IDF (Term Frequency-Inverse Document Frequency)**: 단어의 중요도를 문서 내 빈도와 전체 문서에서의 희소성을 기반으로 계산
2. **Word2Vec**: 단어의 의미적 관계를 포착하는 단어 임베딩 모델
3. **BERT 임베딩**: 문맥을 고려한 고급 언어 모델 기반 임베딩

제목과 본문에 다른 가중치를 적용하여 제목의 중요성을 강조할 수 있습니다. 이는 뉴스 제목이 일반적으로 핵심 내용을 더 잘 요약하기 때문입니다.

## 4. 뉴스 클러스터링

벡터화된 뉴스를 유사한 주제별로 클러스터링합니다.

In [ ]:
# 뉴스 클러스터링 초기화
news_clustering = NewsClustering(
    distance_threshold=0.7,  # 거리 임계값 (낮을수록 더 엄격한 클러스터링)
    min_cluster_size=2,      # 최소 클러스터 크기
    max_cluster_size=20,     # 최대 클러스터 크기
    linkage='average'        # 연결 방법 (평균 연결)
)

news_df = news_clustering.cluster_articles(
    articles_df=news_df, 
    vector_col='vector',
    title_col='Title',
    content_col='Body',
    date_col='Date',
    impact_col='impact_overall'
)

# Now, 'cluster_id' should exist in news_df
cluster_counts = news_df['cluster_id'].value_counts().sort_index()
print("클러스터별 뉴스 수:")
print(cluster_counts)

# 클러스터별 뉴스 수 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=cluster_counts.index, y=cluster_counts.values)
plt.title('Distribution of Clusters')
plt.xlabel('Size of Cluster')
plt.ylabel('Num of Cluster')
plt.show()

2025-03-31 01:36:15,006 - src.models.news_clustering - INFO - Initialized NewsClustering with distance_threshold=0.7, min_cluster_size=2
2025-03-31 01:36:15,006 - src.models.news_clustering - INFO - Clustering 32208 articles


In [ ]:
# 각 클러스터의 대표 키워드 추출
cluster_keywords = news_clustering.get_cluster_keywords(news_vectorizer.vectorizer, news_df)

# 클러스터별 대표 키워드 출력
for cluster_id, keywords in cluster_keywords.items():
    print(f"클러스터 {cluster_id} 키워드: {', '.join(keywords)}")
    
    # 해당 클러스터의 뉴스 제목 몇 개 출력
    cluster_news = news_df[news_df['cluster_id'] == cluster_id]['Title'].head(3).tolist()
    for i, title in enumerate(cluster_news):
        print(f"  - 뉴스 {i+1}: {title}")
    print("-" * 80)

### 뉴스 클러스터링 과정 설명

뉴스 클러스터링은 유사한 내용의 뉴스를 그룹화하는 과정입니다. 이 시스템에서는 다음 방법을 지원합니다:

1. **K-means**: 벡터 공간에서 중심점을 기준으로 데이터를 k개의 클러스터로 나누는 알고리즘
2. **DBSCAN**: 밀도 기반 클러스터링으로, 밀집된 지역을 클러스터로 식별
3. **계층적 클러스터링**: 데이터 포인트 간의 거리를 기반으로 계층적 구조를 형성

클러스터링을 통해 유사한 주제의 뉴스를 그룹화하고, 각 클러스터의 대표 키워드를 추출하여 주제를 파악할 수 있습니다.

## 5. 주가 영향 분석

뉴스가 주가에 미치는 영향을 분석합니다.

In [ ]:
# 주가 데이터 딕셔너리 생성
stock_data = {'005930': stock_df}

# 주가 영향 분석기 초기화
stock_impact_analyzer = StockImpactAnalyzer(
    time_windows=[
        {"name": "immediate", "days": 1},  # 1일 영향
        {"name": "short_term", "days": 3},  # 3일 영향
        {"name": "medium_term", "days": 7}  # 7일 영향
    ],
    impact_thresholds={
        "high": 0.02,    # 2% 가격 변동
        "medium": 0.01,  # 1% 가격 변동
        "low": 0.005     # 0.5% 가격 변동
    }
)

# 뉴스 영향 분석
impact_df = stock_impact_analyzer.analyze_news_impact(news_df, stock_data)

# 영향 분석 결과 확인
impact_columns = [col for col in impact_df.columns if 'impact' in col or 'price_change' in col]
print("영향 분석 결과:")
impact_df[['Title', 'Date'] + impact_columns].head()

In [ ]:
# 영향 점수 분포 시각화
plt.figure(figsize=(12, 6))
sns.histplot(impact_df['impact_score'].dropna(), bins=20, kde=True)
plt.title('뉴스 영향 점수 분포')
plt.xlabel('영향 점수')
plt.ylabel('빈도')
plt.axvline(x=0, color='red', linestyle='--')
plt.show()

### 주가 영향 분석 과정 설명

주가 영향 분석은 뉴스 발행 후 주가 변동을 측정하여 뉴스의 영향력을 평가하는 과정입니다. 다음 단계로 이루어집니다:

1. **뉴스에서 관련 종목 추출**: 뉴스 내용에서 언급된 기업 및 종목 코드 식별
2. **시간 윈도우 정의**: 뉴스 발행 후 영향을 측정할 기간 설정 (즉시, 단기, 중기)
3. **가격 변동 계산**: 각 시간 윈도우에서의 주가 변동 계산
4. **영향 점수 산출**: 가격 변동, 거래량 변화, 변동성 등을 고려한 종합 영향 점수 계산

이 분석을 통해 어떤 뉴스가 주가에 긍정적 또는 부정적 영향을 미쳤는지 파악할 수 있습니다.

## 6. 고급 주가 영향 분석 (수정 및 개선)

개선된 고급 주가 영향 분석기를 사용하여 더 정교한 분석을 수행합니다. 이 분석기는 KO-finbert 감성 분석, 변동성 측정, 시장 상관관계 등을 포함합니다.

In [ ]:
# 고급 주가 영향 분석기 초기화
advanced_analyzer = AdvancedStockImpactAnalyzer(
    stock_data_dir=os.path.join('data', 'stock'),
    use_finbert=True,  # KO-finbert 감성 분석 사용
    use_volatility=True,  # 변동성 분석 사용
    use_market_trend=False,  # 시장 상관관계 분석 (시장 지수 데이터 없음)
    time_window_days=3,  # 3일 시간 윈도우
    impact_threshold=0.02,  # 2% 임계값
    sentiment_weight=0.3  # 감성 가중치
)

In [ ]:
# 고급 주가 영향 분석 수행
advanced_impact_df = advanced_analyzer.analyze_news_impact(news_df, stock_data)

# 분석 결과 확인
print(f"분석된 뉴스 수 (Number of analyzed news): {len(advanced_impact_df)}")
print(f"영향 점수가 계산된 뉴스 수: {advanced_impact_df['impact_overall'].notna().sum()}")

# 감성 분포 확인
sentiment_counts = advanced_impact_df['sentiment_label'].value_counts()
print("\n감성 분포:")
print(sentiment_counts)

In [ ]:
# 영향 점수 결과 확인
impact_columns = ['impact_price', 'impact_sentiment', 'impact_volatility', 'impact_overall']
advanced_impact_df[['Title', 'Date', 'sentiment_label'] + impact_columns].head(10)

In [ ]:
# Ensure Date is datetime and impact_overall is numeric
advanced_impact_df['Date'] = pd.to_datetime(advanced_impact_df['Date'], errors='coerce')
advanced_impact_df['impact_overall'] = pd.to_numeric(advanced_impact_df['impact_overall'], errors='coerce')

# Call the visualization functions
advanced_analyzer.visualize_impact_distribution(advanced_impact_df)
advanced_analyzer.visualize_impact_over_time(advanced_impact_df)

In [ ]:
# 가장 영향력 있는 뉴스 확인
top_positive = advanced_analyzer.get_top_impactful_news(advanced_impact_df, n=5, impact_type='positive')
print("가장 긍정적 영향을 미친 뉴스:")
top_positive[['Title', 'Date', 'impact_overall', 'sentiment_label']]

In [ ]:
# 가장 부정적 영향을 미친 뉴스 확인
top_negative = advanced_analyzer.get_top_impactful_news(advanced_impact_df, n=5, impact_type='negative')
print("가장 부정적 영향을 미친 뉴스:")
top_negative[['Title', 'Date', 'impact_overall', 'sentiment_label']]

### 고급 주가 영향 분석 개선 사항 설명

개선된 고급 주가 영향 분석기는 다음과 같은 향상된 기능을 제공합니다:

1. **강화된 날짜 파싱**: 한국어 뉴스 날짜 형식(YYYYMMDD HH:MM)을 정확히 처리
2. **주가 데이터 처리 개선**: Time 열을 무시하고 일별 데이터에 집중
3. **종목 추출 강화**: 뉴스에서 관련 종목을 더 정확히 추출하고 기본값 제공
4. **KO-finbert 감성 분석**: 한국어 금융 텍스트에 특화된 BERT 모델을 사용한 감성 분석
5. **다중 지표 영향 점수**: 가격 변동, 감성, 변동성을 종합적으로 고려한 영향 점수 계산
6. **강화된 시각화**: 영향 점수 분포 및 시간에 따른 변화를 시각화

이러한 개선을 통해 모든 뉴스에 대해 영향 점수를 계산할 수 있게 되었으며, 더 정확한 분석 결과를 제공합니다.

## 7. 뉴스 추천

클러스터링 및 주가 영향 분석 결과를 기반으로 뉴스를 추천합니다.

In [ ]:
# 뉴스 추천기 초기화
news_recommender = NewsRecommender(
    weights={
        'impact': 0.4,       # 주가 영향
        'quality': 0.2,      # 질 평가
        'content': 0.3,      # 콘텐츠 기반
        'collaborative': 0.0,  # 개인화 없음
        'recency': 0.1       # 최신 뉴스
    }
)

In [ ]:
# 고급 영향 분석 결과를 병합
merged_df = news_df.copy()
merged_df['impact_score'] = advanced_impact_df['impact_overall']
merged_df['sentiment_label'] = advanced_impact_df['sentiment_label']

# 뉴스 추천 수행
recommended_news = news_recommender.recommend_articles(merged_df, top_n=10)

# 추천 결과 확인
print(f"추천된 뉴스 수: {len(recommended_news)}")
recommended_news[['Title', 'Date', 'impact_score', 'sentiment_label', 'recommendation_score']].head(10)

### 뉴스 추천 과정 설명

뉴스 추천은 네이버의 AiRS 시스템에서 영감을 받아 다음 메커니즘을 조합합니다:

1. **협업 필터링 (CF)**: 유사한 사용자의 선호도를 기반으로 추천 (개인화 없으므로 가중치 0)
2. **콘텐츠 기반 필터링 (CBF)**: 뉴스 내용의 유사성을 기반으로 추천
3. **질의 확장 (QE)**: 검색 쿼리를 확장하여 관련 뉴스 추천
4. **주가 영향 (SI)**: 주가에 큰 영향을 미친 뉴스 우선 추천
5. **최신 뉴스 (Latest)**: 최근 발행된 뉴스 우선 추천

각 메커니즘에 가중치를 적용하여 최종 추천 점수를 계산합니다. 이 시스템에서는 개인화 대신 주가 영향에 높은 가중치를 부여합니다.

## 8. 결과 시각화 및 평가

추천 시스템의 결과를 시각화하고 평가합니다.

In [ ]:
# 클러스터와 영향 점수 관계 시각화
plt.figure(figsize=(12, 6))
sns.boxplot(x='cluster_id', y='impact_score', data=merged_df)
plt.title('Stock Impact Distribution by Cluster')
plt.xlabel('Cluster ID')
plt.ylabel('Impact Score')
plt.axhline(y=0, color='red', linestyle='--')
plt.show()

In [ ]:
# 감성과 영향 점수 관계 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='sentiment_label', y='impact_score', data=merged_df)
plt.title('Stock Impact Score by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Impact Score')
plt.axhline(y=0, color='red', linestyle='--')
plt.show()

In [ ]:
# 시간에 따른 클러스터 분포 시각화
# 날짜를 datetime으로 변환
if not pd.api.types.is_datetime64_any_dtype(merged_df['Date']):
    merged_df['Date'] = merged_df['Date'].apply(lambda x: pd.to_datetime(str(x).split()[0], format='%Y%m%d'))

# 월별로 그룹화
merged_df['month'] = merged_df['Date'].dt.to_period('M')
cluster_by_month = pd.crosstab(merged_df['month'], merged_df['cluster_id'])

# 시각화
plt.figure(figsize=(14, 8))
cluster_by_month.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('월별 클러스터 분포')
plt.xlabel('월')
plt.ylabel('뉴스 수')
plt.legend(title='클러스터 ID')
plt.xticks(rotation=45)
plt.show()

### 시스템 평가

CLEAR 시스템의 성능을 평가합니다. 실제 레이블이 없으므로 내부 일관성 및 분포 특성을 기반으로 평가합니다.

In [ ]:
# 평가기 초기화
evaluator = CLEAREvaluator()

# 클러스터링 품질 평가
clustering_scores = evaluator.evaluate_clustering(merged_df)

print("클러스터링 평가 결과:")
for metric, score in clustering_scores.items():
    print(f"{metric}: {score:.4f}")

In [ ]:
# # 영향 분석 일관성 평가
# impact_consistency = evaluator.evaluate_impact_consistency(merged_df)

# print("\n영향 분석 일관성 평가:")
# print(f"감성-영향 일관성 점수: {impact_consistency:.4f}")

In [ ]:
# # 추천 다양성 평가
# diversity_score = evaluator.evaluate_recommendation_diversity(
#     recommended_df=recommended_news,
#     cluster_col='cluster_id'
# )

# print("\n추천 다양성 평가:")
# print(f"클러스터 다양성 점수: {diversity_score:.4f}")

## 결론

CLEAR 시스템은 네이버의 AiRS 아키텍처를 기반으로 하되, 개인화 대신 주가 영향에 중점을 둔 뉴스 추천 시스템입니다. 이 시스템은 다음과 같은 주요 구성 요소로 이루어져 있습니다:

1. **텍스트 전처리**: 한국어 뉴스 텍스트를 분석에 적합한 형태로 변환
2. **뉴스 벡터화**: 텍스트를 수치 벡터로 변환하여 유사성 계산 가능
3. **뉴스 클러스터링**: 유사한 주제의 뉴스를 그룹화
4. **주가 영향 분석**: 뉴스가 주가에 미치는 영향 측정
5. **고급 주가 영향 분석**: KO-finbert 감성 분석, 변동성 측정 등을 통한 정교한 분석
6. **뉴스 추천**: 클러스터링 및 주가 영향을 고려한 뉴스 추천

이 시스템은 금융 뉴스를 분석하고 주가에 영향을 미치는 중요한 뉴스를 식별하는 데 유용하게 활용될 수 있습니다.